In [54]:
#S-> ABC | C
#A-> a | bB | epsilon
#B->p | epsilon
#C-> c

In [55]:
Non_terminals=["S","A","B","C"]
Terminals=["a","b","c","p","$"]

In [56]:
production_rules={"S":["ABC","C"],"A":["a","bB","@"],"B":["p","@"],"C":["c"]}

In [57]:
def First(symbol):
  first=set([])
  #if symbol is terminal
  if(symbol[0] in Terminals or symbol=="@"):
    first.add(symbol[0])
    return first
  #symbol length>1 (ABC)
  if len(symbol)>1:
    for sym in symbol:
      first2=First(sym)
      if '@' in first2:
        first=first.union(first2-{'@'})
      else:
        first=first.union(first2)
  
  else:
    for production in production_rules[symbol[0]]:
      if(production[0] in Terminals):
        first.add(production[0])
      elif(production[0]=="@"):
        first.add(production[0])
      elif(production[0] in Non_terminals):
          for string in production:
            first2=First(string)
            if("@" in first2):
              first=first.union((first2-{"@"}))
            else:
              first=first.union(first2)
              break
  return first

In [58]:
print("First for each Non-Terminals are:")
for nt in Non_terminals:
  print(f'{nt} : {First(nt)}')


First for each Non-Terminals are:
S : {'p', 'a', 'b', 'c'}
A : {'@', 'a', 'b'}
B : {'@', 'p'}
C : {'c'}


In [59]:
def get_key(val):
    keys=set([])
    for key, value in production_rules.items():
        for string in value:
          for letter in string:
           if val == letter:
             keys.add(key)
    return keys
 

In [60]:
print(get_key("a"))

{'A'}


In [61]:
def Follow(symbol):
  follow=set([])
  if(symbol=="S"):
    follow.add("$")
  keys=get_key(symbol)
  for k in keys:
    for production in production_rules[k]:
      for i in range(0,len(production)):
          if production[i]==symbol :
            j=i
            if j!=len(production)-1:
              for j in range(i,len(production)):
                first=First(production[j+1])
                if('@' in  first):
                  follow=follow.union(Follow(k))
                  follow=follow.union((first-{'@'}))
                else:
                  follow=follow.union(first)
                  break
            elif(production[i]==symbol and i==len(production)-1):
                follow=follow.union(Follow(k))
            elif(production[i]==symbol and symbol==k):
                return
            
  return follow    

  

In [62]:
def make_Table():  #Make first and follow table
  table_dict=dict({})
  for nt in Non_terminals:
    table_dict[nt]=[First(nt),Follow(nt)]
  return table_dict


In [72]:
print("Non-Terminals\t \t Follow")
table=make_Table()
for k in table.keys():
  print(f'{k}\t\t\t  {table[k][1]}')

Non-Terminals	 	 Follow
S			  {'$'}
A			  {'p', 'c', '$'}
B			  {'p', 'c', '$'}
C			  {'$'}


In [64]:
def LL1Table():  #make LL(1) table
  S={}
  A={}
  B={}
  C={}
  ll={"S":S,"A":A,"B":B,"C":C}

  for k in ll.keys():
    for t in Terminals:
      ll[k][t]=list([])
  table=make_Table()
  for nt in Non_terminals:
    for prod in production_rules[nt]:
      first=First(prod)
      for f in first:
        if f=='@':
         for fol in table[nt][1]:
          ll[nt][fol].append(f'{nt}->epsilon')
        elif(f in table[nt][0]):  
          ll[nt][f].append(f'{nt}->{prod}')
  for k in ll.keys():
    ll[k]=dict(sorted(ll[k].items()))
    print(f'{k}:=\t',end='')
    for s in ll[k]:
      print(f'{s}: {ll[k][s]}\t',end='')
    print("\n")

In [65]:
print("LL(1) Parsing Table:\n")
print("Non-terminals")
LL1Table()

LL(1) Parsing Table:

Non-terminals
S:=	$: []	a: ['S->ABC']	b: ['S->ABC']	c: ['S->ABC', 'S->C']	p: ['S->ABC']	

A:=	$: ['A->epsilon']	a: ['A->a']	b: ['A->bB']	c: ['A->epsilon']	p: ['A->epsilon']	

B:=	$: ['B->epsilon']	a: []	b: []	c: ['B->epsilon']	p: ['B->p', 'B->epsilon']	

C:=	$: []	a: []	b: []	c: ['C->c']	p: []	



In [66]:
#Since there are two instances where there are more than one productions in one slot, this grammar isn't LL(1).